*This jupyter notebook is part of Arizona State University's course CAS 503 (Fundamentals of Complex Systems Science: Collectives) and was written by Bryan Daniels.  It was last updated August 24, 2023.*

*Data used in this notebook comes from the following publication:*
* *McPherson, E. Gregory; van Doorn, Natalie S.; Peper, Paula J. 2016. Urban tree database. Fort Collins, CO: Forest Service Research Data Archive. Updated 21 January 2020. https://doi.org/10.2737/RDS-2016-0005*

# Identifying scaling laws

In lecture, we saw how scaling laws can often be used to relate quantities in collectives to one another.  A common basic type of scaling takes the form of a "self-similar" power law:
$$
A = k B^\beta,
$$
where $A$ and $B$ are two quantities of interest, $k$ is a constant factor, and $\beta$ is the exponent of the power law.

In this exercise, we'll look for scaling laws within some real-world ecological data from measurements of trees.

First import some useful standard python packages:

In [ ]:
import numpy
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 18}) # increases font size on plots
import pandas
from pathlib import Path # to handle file paths across all operating systems

`numpy` (Numerical Python) is very useful for working with arrays of numbers; we'll use `matplotlib` to make plots; and `pandas` is a powerful package for managing data—think of it as a grown-up version of an Excel spreadsheet.

Now we'll use `pandas` to load in some public tree data from a CSV file provided by the Forest Service's "Urban Tree Database":

*(Hint: Are you getting a `FileNotFoundError` when running the following line?  If so, be sure that you have downloaded the GitHub repository in its entirety before trying to run this notebook.  See the [instructions here](https://github.com/bcdaniels/CAS-503-Collectives#setting-up-this-git-repository) for more information.)*

In [ ]:
data = pandas.read_csv(Path('data/McPhersonEtAl2020/TS3_Raw_tree_data.csv'))

What is this `data` object?

In [ ]:
type(data)

It's a `pandas` `DataFrame` object, basically a spreadsheet of data values.  To see a preview of the data, use the `head` function:

In [ ]:
data.head()

Each row corresponds to a single tree.  How many trees do we have?

In [ ]:
len(data)

A lot of trees!  We can also see a list of all the column names:

In [ ]:
data.columns

Now we'll use the following syntax to look at a subset of the data coming from a single species, the [Japanese zelkova](https://en.wikipedia.org/wiki/Zelkova_serrata):

In [ ]:
dataSubset = data[ (data['CommonName']=='Japanese zelkova') ]

And we can, for instance, look at the cities and tree heights for the 112 Japanese zelkova in the database:

In [ ]:
dataSubset[['City','TreeHt (m)']]

Another common measurement for those who study trees is the "diameter at breast height" (DBH).  This measures the diameter of a tree's trunk at a fixed distance from the ground.

❓**Naively (or not so naively, if you are a tree biologist), how would you expect the height of a tree to scale with the diameter of its trunk?  Or would you expect any kind of scaling relationship at all?** *Hint: There's no wrong answer here—we're just practicing the good scientific habit of guessing what the answer will be before you calculate it.*

✳️ **Answer:** 

Now let's check the data.

The function `plt.scatter` will take two columns of our dataset and plot points in a 2D scatter plot:

In [ ]:
plt.scatter(dataSubset['DBH (cm)'],dataSubset['TreeHt (m)'])
plt.xlabel('diameter (cm)')
plt.ylabel('height (m)');

There is definitely some kind of relationship.  Maybe linear?  Maybe there is a curve?  Recall that, in lecture, we saw how a log-log plot will show a power law relationship as a straight line.  

❓**Plot the same data on a log-log plot.**  *Hints: `numpy.log` will take the logarithm of each value in an array of data: e.g. `numpy.log(dataSubset['DBH (cm)'])`.  Don't forget axes labels!*

In [ ]:
# ✳️ Answer

This looks quite linear, indicating that tree height does scale (at least roughly) as a power law of diameter.

Recall that the slope of a line on a log-log plot indicates the power of the power law: taking the logarithm of both sides of the equation at the beginning of the notebook, we have
$$
\log A = \log k + \beta \log B,
$$
so the exponent $\beta$ in the original equation is the slope of a linear relationship between $\log A$ and $\log B$.

So we will now fit a line to our logged data, making use of a polynomial fitting function from `numpy` (a polynomial of degree 1 is a line):

In [ ]:
polyfit = numpy.polynomial.Polynomial.fit(numpy.log(dataSubset['DBH (cm)']),
                                          numpy.log(dataSubset['TreeHt (m)']),1).convert()
polyfit

This output means that the logarithm of tree height is approximately 0.39 plus 0.57 times the logarithm of DBH.

We can plot the fit line using the following code:

In [ ]:
logDiameters = numpy.linspace(0,5)
predictedLogHeights = 0.39096 + 0.57336*logDiameters
plt.plot(logDiameters,predictedLogHeights,color='orange')
plt.xlabel('log diameter (cm)')
plt.ylabel('log height (m)');

❓**Combine your scatter plot of the logged data above with the fit line to display them overlapping on the same plot.** *Hint: You can typically make overlapping plots simply by calling plotting functions one after another.*

In [ ]:
# ✳️ Answer

❓**What is our estimate of the exponent $\beta$ describing the scaling of tree height with diameter? Is the scaling linear, superlinear, or sublinear?**

✳️ **Answer:** 

❓**On the original, non-logged scatter plot, if we overlay the fit scaling law, should it curve upward as diameter increases, curve downward, or appear as a straight line?**

✳️ **Answer:** 

Check your answer with the following code that adds the power law fit to the original scatter plot:

In [ ]:
plt.scatter(dataSubset['DBH (cm)'],
            dataSubset['TreeHt (m)'])

logDiameters = numpy.linspace(0,5)
diameters = numpy.exp(logDiameters)
predictedLogHeights = 0.39096 + 0.57336*logDiameters
predictedHeights = numpy.exp(predictedLogHeights)

plt.plot(diameters,predictedHeights,color='orange')
plt.xlabel('diameter (cm)')
plt.ylabel('height (m)');

❓**Extrapolate using your fit scaling law to predict the height of a hypothetical Japanese zelkova that has a trunk 200 cm in diameter.**

✳️ **Answer:** 

⚛️ **Bonus question (for nothing but bragging rights):** An important quantity for ecologists is the bio-mass, basically the total mass of the wood in our tree example (which is related, for example, to the total amount of carbon captured by a forest).  The bio-mass of a tree is harder to measure than, say, its diameter.  Use your scaling law and any needed basic assumptions to predict how a Japanese zelkova's biomass scales as a function of its diameter.  This relates our scaling analysis with ecologically relevant collective effects of the cells that combine to form a tree.

✴️ **Answer:** 